In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
x = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], requires_grad=True)
y = F.dropout(x, p=0.7, training=True)
print(y)


tensor([0.0000, 3.3333, 0.0000, 3.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 3.3333, 0.0000, 3.3333, 0.0000, 3.3333, 0.0000, 0.0000, 3.3333,
        0.0000], grad_fn=<MulBackward0>)


In [8]:
# cfg = [32*32*2,200,200,200,200,200,200,10]
cfg = [784,200,200,10]
print('layers:',cfg,'\n')

feedforward_ADDs = 0
for i in range(len(cfg)):
    if i < len(cfg)-1:
        feedforward_ADDs += cfg[i]*cfg[i+1]
print(f'feedforward_ADDs {feedforward_ADDs:,}')

print(f'')
print(f'')
print(f'')



BP_MACs = 0
for i in range(len(cfg)):
    if i < len(cfg)-2:
        BP_MACs += cfg[i+1]*cfg[i+2]
print(f'BP_MACs {BP_MACs:,}')

BP_emuls = 0
for i in range(len(cfg)):
    if i < len(cfg)-1:
        BP_emuls += cfg[i+1]
print(f'BP_emuls {BP_emuls:,}')

BP_eadds = 0
for i in range(len(cfg)):
    if i < len(cfg)-1:
        BP_eadds += cfg[i]*cfg[i+1]
print(f'BP_eadds {BP_eadds:,}')

BP_total_ops = BP_MACs*2 + BP_emuls + BP_eadds
print(f'-->BP_total_ops {BP_total_ops:,}  (MAC = 2ops)')

print(f'')


DFA_MACs = 0
for i in range(len(cfg)):
    if i < len(cfg)-2:
        DFA_MACs += cfg[i+1]*cfg[-1]
print(f'DFA_MACs {DFA_MACs:,}')

DFA_emuls = 0
for i in range(len(cfg)):
    if i < len(cfg)-1:
        DFA_emuls += cfg[i+1]
print(f'DFA_emuls {DFA_emuls:,}')

DFA_eadds = 0
for i in range(len(cfg)):
    if i < len(cfg)-1:
        DFA_eadds += cfg[i]*cfg[i+1]
print(f'DFA_eadds {DFA_eadds:,}')

DFA_total_ops = DFA_MACs*2 + DFA_emuls + DFA_eadds
print(f'-->DFA_total_ops {DFA_total_ops:,}  (MAC = 2ops)')

print(f'')
print(f'')
print(f'')

BP_plus_FF_ops = BP_total_ops + feedforward_ADDs
print(f'BP_plus_FF_ops {BP_plus_FF_ops:,}')

DFA_plus_FF_ops = DFA_total_ops + feedforward_ADDs
print(f'DFA_plus_FF_ops {DFA_plus_FF_ops:,}')


print(f'')
print(f'')
print(f'')

print(f'=====focus on ratio=======')
DFA_FF_OVER_BP_FF = DFA_plus_FF_ops / BP_plus_FF_ops
print(f'DFA_FF_OVER_BP_FF = {DFA_FF_OVER_BP_FF*100:.2f}%')

DFA_FF_OVER_BP_FF_only_MACsANDADDs = ((DFA_MACs*2) + feedforward_ADDs) / ((BP_MACs*2) + feedforward_ADDs)
print(f'DFA_FF_OVER_BP_FF_only_MACsANDADDs = {DFA_FF_OVER_BP_FF_only_MACsANDADDs*100:.2f}%')





print(f'')
print(f'')
print(f'')





print(f'=====focus on ratio at only backward=======')

DFA_OVER_BP = DFA_total_ops / BP_total_ops
print(f'DFA_OVER_BP = {DFA_OVER_BP*100:.2f}%')

DFA_OVER_BP_only_MACs = (DFA_MACs*2) / (BP_MACs*2)
print(f'DFA_OVER_BP_only_MACs = {DFA_OVER_BP_only_MACs*100:.2f}%')

layers: [784, 200, 200, 10] 

feedforward_ADDs 198,800



BP_MACs 42,000
BP_emuls 410
BP_eadds 198,800
-->BP_total_ops 283,210  (MAC = 2ops)

DFA_MACs 4,000
DFA_emuls 410
DFA_eadds 198,800
-->DFA_total_ops 207,210  (MAC = 2ops)



BP_plus_FF_ops 482,010
DFA_plus_FF_ops 406,010



=====focus on ratio=======
DFA_FF_OVER_BP_FF = 84.23%
DFA_FF_OVER_BP_FF_only_MACsANDADDs = 73.13%



=====focus on ratio at only backward=======
DFA_OVER_BP = 73.16%
DFA_OVER_BP_only_MACs = 9.52%
